# Lab

Using the flights dataframe answer the following questions

1. When do the first and last flights leave each day?

2. When there is a missing value in `dep_time` then this is an indication of canceled flight. Find the number of cancelled flights for each (`carrier`, `origin`) combination.

3. Which carrier has the worst `dep_delay`s? 

4. Which plane (`tailnum`) has the worst on-time record?

5. For each plane, count the number of flights before the first delay of greater than 1 hour.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/mguner/teaching/main/datasets/nyc_flights/nyc_flights.csv')

In [4]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [68]:
#Q1- first flight
df['month-day'] = df['month'].astype(str)+df['day'].astype(str)
# finding minimum dep_time and getting the value into filter
idmin = df.groupby(['month-day'])['dep_time'].transform(min) == df['dep_time']
df[idmin][['month-day','dep_time','flight']]

,month-day,dep_time,flight
0,11,517.0,1545
842,12,42.0,707
1785,13,32.0,707
2699,14,25.0,707
3614,15,14.0,739
...,...,...,...
332195,926,451.0,1877
333191,927,453.0,1877
334187,928,451.0,1877
334869,929,521.0,322


In [69]:
#Q1- last flight
# finding maximun dep_time and getting the value into filter
idmax = df.groupby(['month-day'])['dep_time'].transform(max) == df['dep_time']
df[idmax][['month-day','dep_time','flight']]

,month-day,dep_time,flight
837,11,2356.0,727
1776,12,2354.0,727
2688,13,2349.0,739
3606,14,2358.0,727
3607,14,2358.0,739
...,...,...,...
333187,926,2349.0,745
334182,927,2346.0,745
334865,928,2351.0,745
335779,929,2356.0,745


In [70]:
#Q2
#selecting null values from dep_time and counting the values by grouping them by carrier and origin
df[df['dep_time'].isna()][['carrier','origin']].groupby(['carrier','origin']).agg({'carrier':'count'})

carrier
carrier origin         
9E      EWR          68
        JFK         807
        LGA         169
AA      EWR          99
        JFK         141
        LGA         396
AS      EWR           2
B6      EWR          74
        JFK         315
        LGA          77
DL      EWR          39
        JFK         100
        LGA         210
EV      EWR        2164
        JFK          82
        LGA         571
F9      LGA           3
FL      LGA          73
MQ      EWR         168
        JFK         327
        LGA         739
OO      LGA           3
UA      EWR         435
        JFK          44
        LGA         207
US      EWR          75
        JFK          26
        LGA         562
VX      EWR          10
        JFK          21
WN      EWR         105
        LGA          87
YV      LGA          56

In [27]:
#Q3
# taking mean of dep_delay's to find the worst of them
df.groupby('carrier').agg({'dep_delay':'mean'}).sort_values(by='dep_delay').idxmax()

dep_delay    F9
dtype: object

In [50]:
#Q4:
# taking mean of arr_delay & dep_delay
df2 = df.groupby('tailnum').agg({'dep_delay':'mean','arr_delay':'mean'})
# adding both delays to get the worst on-time record for flight
df2['worst_on_time'] = df2['dep_delay']+df2['arr_delay']
df2['worst_on_time'].idxmax()

'N844MH'

In [65]:
#Q5:
# grouped the df by flight and used apply function to count the values untill certain condition
df.groupby('flight')['dep_delay'].apply(lambda x: (x == 60).count()).reset_index(name='count')

,flight,count
0,1,701
1,2,51
2,3,631
3,4,393
4,5,324
...,...,...
3839,6171,1
3840,6177,164
3841,6180,6
3842,6181,80
